In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.wkt import loads
import os
import sys

script_path = os.path.abspath(os.path.join("..", "scripts"))
sys.path.append(script_path)

import analysis_functions
import plot_functions

### Load data

In [ ]:
##### LOAD DATA FROM CSV (saved from BigQuery) #####
df = pd.read_csv("../data/shadowtankers.csv")
df['dt'] = pd.to_datetime(df['dt'], format="mixed")
df['geometry'] = df['geometry'].apply(loads)
gdf = gpd.GeoDataFrame(df, geometry='geometry', crs="EPSG:4326")

### Identify "dark" periods

In [ ]:
# Define threshold for hours dark
dark_hours_threshold = 5

In [ ]:
dark_periods_df = analysis_functions.identify_dark_periods(gdf, dark_hours_threshold) 

In [ ]:
dark_periods_df.head(2)

In [ ]:
# Export product
dark_periods_df.to_csv("../products/1_darkvoyages/shadow_tanker_dark_periods.csv")

### Identify periods where two proximally-located vessels are dark simultaneously

In [ ]:
# Define h3 level for proximity matching (could be improved later using max ship speed)
h3_level = 4

In [ ]:
proximity_df = analysis_functions.find_proximity_dark_periods(dark_periods_df, h3_level)

In [ ]:
proximity_df.head(2)

In [ ]:
proximity_df.to_csv("../products/1_darkvoyages/shadow_tanker_colocated_dark_periods.csv")

### Plot dark periods locations

In [ ]:
heatmap_df = dark_periods_df.groupby('h3_lv3_start').size().reset_index(name='total_dark_periods')

fig, ax = plot_functions.plot_h3_heatmap(
    df=heatmap_df,
    h3_column="h3_lv3_start",
    heatmap_column="total_dark_periods"
)

plt.title("Shadow Tankers Dark Periods (>5 hrs) Heatmap")
plt.gcf().canvas.draw_idle()
plt.savefig("../products/1_darkvoyages/shadowtankers_dark_periods_heatmap.jpg", dpi=100, bbox_inches="tight")

In [ ]:
heatmap2_df = proximity_df.groupby('h3_end_1').size().reset_index(name='total_proximal_dark_periods')

fig, ax = plot_functions.plot_h3_heatmap(
    df=heatmap2_df,
    h3_column="h3_end_1",
    heatmap_column="total_proximal_dark_periods"
)

plt.title("Shadow Tankers Collocated Dark Periods Heatmap")
plt.gcf().canvas.draw_idle()
plt.savefig("../products/1_darkvoyages/shadowtankers_colocated_dark_periods_heatmap.jpg", dpi=100, bbox_inches="tight")

In [ ]:
import importlib
importlib.reload(plot_functions)

### Dark colocation vignette

In [ ]:
vignette_row = proximity_df.iloc[58]
mmsi_list = [vignette_row["mmsi_1"], vignette_row["mmsi_2"]]

start_time = min(vignette_row["dt_start_1"],vignette_row["dt_start_2"])
end_time = max(vignette_row["dt_end_1"],vignette_row["dt_end_2"])

# Generate the vignette plot using the full GeoDataFrame (vessel_gdf)
fig, ax = plot_functions.generate_vignette_plot(gdf, mmsi_list, start_time, end_time, buffer_hrs=1)

plt.title("Shadow Tankers Colocation Vignette")
plt.gcf().canvas.draw_idle()
plt.savefig("../products/1_darkvoyages/shadowtankers_colocation_vignette.jpg", dpi=100, bbox_inches="tight")